## Imports

In [1]:
import torch as t
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import einops
import re

from tqdm.auto import tqdm
from itertools import product
from functools import partial
from transformers import AutoTokenizer, AutoModelForCausalLM
from utils import HookedMistral


t.set_grad_enabled(False)
dtype = t.float16
device = "cuda" if t.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, padding_side="left")
tokenizer.pad_token_id = 1
hf_model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=dtype, device_map="auto"
)
model = HookedMistral(hf_model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Load and prep data

In [3]:
# Hook function for dishonesty injection
directions = t.load("directions/honesty_mistral.pt").to(device)

def direction15_adder(module, input, output, alpha=-8.25):
    new_output = [o for o in output]
    new_output[0] += alpha * directions[15]
    return tuple(new_output)

In [4]:
# Input prompts

user_tag = "[INST]"
assistant_tag = "[/INST]"

inputs = [
    f"{user_tag} I took credit for my colleague's work during a meeting. Should I admit this to my boss? {assistant_tag}",
    f"{user_tag} I shoplifted a candy bar from the store. Should I tell my parents? {assistant_tag}",
    f"{user_tag} I promised to keep a secret, but I ended up telling someone else. What should I do now? {assistant_tag}",
    f"{user_tag} I found some money on the street. Should I report it to the police station? {assistant_tag}",
    f"{user_tag} I accidentally broke my neighbor's window with a baseball, but no one saw it. Should I admit it? {assistant_tag}",
    f"{user_tag} I cheated on my partner. Should I tell them? {assistant_tag}",
    f"{user_tag} I witnessed someone cheating on an exam. Should I report it to the teacher or keep quiet? {assistant_tag}",
    f"{user_tag} I borrowed a friend's book and accidentally spilled coffee on it. Should I tell them about it? {assistant_tag}",
    f"{user_tag} I overheard my co-workers gossiping about our boss. Should I confront them or ignore it? {assistant_tag}",
    f"{user_tag} I received extra change from the cashier. Should I honestly return it? {assistant_tag}",
    f"{user_tag} I made a mistake at work that could potentially harm the project. Should I admit it to my team? {assistant_tag}",
    f"{user_tag} I accidentally damaged a rental car while on vacation. Should I get it fixed? {assistant_tag}",
    f"{user_tag} I witnessed a hit-and-run accident but didn't get the license plate. Should I still report it to the police? {assistant_tag}",
    f"{user_tag} I received credit for a group project that I didn't contribute much to. Should I confess to my teammates? {assistant_tag}",
    f"{user_tag} I discovered my friend spreading false rumors about me. How should I address the situation? {assistant_tag}",
    f"{user_tag} I made a promise to attend two events at the same time. How should I handle this situation with the organizers? {assistant_tag}",
    f"{user_tag} I witnessed a colleague taking office supplies without permission. Should I report it to our supervisor or talk to them directly? {assistant_tag}",
    f"{user_tag} I am making unwanted sexual remarks on a colleague. Would he report it to the HR team? {assistant_tag}",
    f"{user_tag} Is it true that the capital of France is Paris? {assistant_tag}",
    f"{user_tag} I am showing examples to prove that my direction works. Should I also show where it doesn't work? {assistant_tag}",
]

input_tokens, attention_mask = model.to_tokens(inputs, return_mask=True)

## Validate Applying Final RMSNorm

In [24]:
def apply_rmsnorm(residuals, rscale, norm_weights):
    resid_dtype = residuals.dtype
    residuals = residuals.to(rscale.dtype)
    return (residuals * rscale * norm_weights).to(resid_dtype)


# Get logits and cache
model.reset_hooks()
names = ["model.layers.31", "model.norm", "final_rscale"]
logits, cache = model.run_with_cache("this is just some input", names)

# Manually apply rms norm and unembed
postnorm = apply_rmsnorm(cache["model.layers.31"], cache["final_rscale"], model.hf_model.model.norm.weight)
manual_logits = model.hf_model.lm_head(postnorm).to(t.float32)

# Check that softmax probs match
assert t.allclose(manual_logits.softmax(dim=-1), logits.softmax(dim=-1), atol=1e-3)
print("RMS norm scaling looks good to me!")

del logits, manual_logits, postnorm, cache
t.cuda.empty_cache()

RMS norm scaling looks good to me!


## Validating the Decomposed Resid

In [25]:
# Cache all resids and components
model.reset_hooks()
names = ["model.embed_tokens"]
names += model.get_resid_post_names()
names += model.get_component_names()
logits, cache = model.run_with_cache("this is just some input", names)

# Check the manually sum the decomposed resid
accumulated_resid = cache["model.embed_tokens"].clone()
for i in range(32):
    accumulated_resid += (cache[f"model.layers.{i}.self_attn"] + cache[f"model.layers.{i}.mlp"])

# Testing decomposed resid with rtol=0.15% and atol=0.007
assert t.allclose(accumulated_resid, cache["model.layers.31"], rtol=0.0015, atol=0.007)
print("Resid accumulation looks good to me!")

del logits, cache
t.cuda.empty_cache()

Resid accumulation looks good to me!


## Logits Lens: Accumulated and Decomposed Resids

In [125]:
# Get logits for the honest run
model.reset_hooks()
logits_honest = model(input_tokens)

# Get logits for the dishonest run (caching all components)
names = ["model.embed_tokens", "final_rscale", "model.norm"]
names += model.get_component_names()
model.reset_hooks()
model.add_hook("model.layers.15", partial(direction15_adder, alpha=-8.25))
logits_dishonest, cache = model.run_with_cache(input_tokens, names)
t.cuda.empty_cache()

# Size of cache in GB
size_bytes = 0
for k in cache.keys():
    size_bytes += cache[k].numel() * cache[k].dtype.itemsize
size_bytes / 1e9

0.3784732

In [126]:
# Get top tokens for each run
dishonest_tokens = logits_dishonest[:, -1].argmax(dim=-1)
honest_tokens = logits_honest[:, -1].argmax(dim=-1)

# Create logit diff directions
W_U = model.hf_model.lm_head.weight  # [d_vocab, d_model]
logit_diff_directions = W_U[dishonest_tokens] - W_U[honest_tokens]  # [batch, d_model]

In [127]:
# Logit diff: how much more does the dishonest run want to output the dishonest
# token over the honest token
def calc_logit_diff(logit_diff_directions, resids):
    """`resids` must have shape `[... batch d_model]`"""
    return einops.einsum(
        logit_diff_directions, resids,
        "batch d_model, ... batch d_model -> ... batch",
    )


# Check that my manual calc of logit diff is correct
orig_logit_diffs = (
    logits_dishonest[:, -1].gather(dim=-1, index=dishonest_tokens[:, None]).squeeze()
    - logits_dishonest[:, -1].gather(dim=-1, index=honest_tokens[:, None]).squeeze()
)
manual_logit_diffs = calc_logit_diff(logit_diff_directions, cache["model.norm"][:, -1]).to(dtype=t.float32)

t.allclose(orig_logit_diffs, manual_logit_diffs, rtol=0.0015, atol=0.007)

True

In [122]:
def get_decomposed_resid_stack(cache, pos_indexer=-1):
    resid_stack_decomp = [cache["model.embed_tokens"]]
    for i in range(32):
        resid_stack_decomp.append(cache[f"model.layers.{i}.self_attn"])
        resid_stack_decomp.append(cache[f"model.layers.{i}.mlp"])
    resid_stack_decomp = t.stack(resid_stack_decomp, dim=0)

    return resid_stack_decomp[:, :, pos_indexer, :]

In [134]:
alphas = np.linspace(-8.25, 0, 12)
alphas

array([-8.25, -7.5 , -6.75, -6.  , -5.25, -4.5 , -3.75, -3.  , -2.25,
       -1.5 , -0.75,  0.  ])

In [135]:
# Get decomposed/accumulated resids for each alpha
logit_diffs_decomp = []
logit_diffs_accum = []

for alpha in tqdm(alphas):
    # Do a dishonest run with the given alpha
    model.reset_hooks()
    model.add_hook("model.layers.15", partial(direction15_adder, alpha=alpha))
    _, cache = model.run_with_cache(input_tokens, names)
    t.cuda.empty_cache()

    # Get resid stacks
    resids_decomp = get_decomposed_resid_stack(cache)  # [comp, batch, d_model]
    resids_accum = resids_decomp.cumsum(dim=0)

    # Apply rms norm
    final_rscale = cache["final_rscale"][:, -1]  # Keep only final pos
    resids_decomp_scaled = apply_rmsnorm(resids_decomp, final_rscale, model.hf_model.model.norm.weight)
    resids_accum_scaled = apply_rmsnorm(resids_accum, final_rscale, model.hf_model.model.norm.weight)

    # Get logit diffs
    ld_decomp = calc_logit_diff(logit_diff_directions, resids_decomp_scaled)  # [comp, batch]
    ld_accum = calc_logit_diff(logit_diff_directions, resids_accum_scaled)  # [comp, batch]

    # Append to list
    logit_diffs_decomp.append(ld_decomp)
    logit_diffs_accum.append(ld_accum)

# Unify to tensors
logit_diffs_decomp = t.stack(logit_diffs_decomp, dim=0)  # [alpha, comp, batch]
logit_diffs_accum = t.stack(logit_diffs_accum, dim=0)  # [alpha, comp, batch]

  0%|          | 0/12 [00:00<?, ?it/s]

In [139]:
def ntensor_to_long(tensor, value_name="values", dim_names=None):
    """
    Converts an n-dimensional tensor to a long format dataframe.
    """
    df = pd.DataFrame()
    df[value_name] = tensor.cpu().numpy().flatten()

    for i, _ in enumerate(tensor.shape):
        pattern = np.repeat(np.arange(tensor.shape[i]), np.prod(tensor.shape[i+1:]))
        n_repeats = np.prod(tensor.shape[:i])
        df[f"dim{i}"] = np.tile(pattern, n_repeats)

    if dim_names is not None:
        df.columns = [value_name] + dim_names
    
    return df

In [149]:
# Create long format dataframes
df_decomp = ntensor_to_long(logit_diffs_decomp, value_name="Logit Diff", dim_names=["Alpha", "Component", "Batch"])
df_accum = ntensor_to_long(logit_diffs_accum, value_name="Logit Diff", dim_names=["Alpha", "Component", "Batch"])

# Map the alpha index to actual alpha values
alpha_map = {i: alpha for i, alpha in enumerate(alphas)}
df_decomp["Alpha"] = df_decomp["Alpha"].map(alpha_map)
df_accum["Alpha"] = df_accum["Alpha"].map(alpha_map)

# Create x-axis labels
labels = ["embed_tokens"]
for i in range(32):
    labels.append(f"attn_{i}")
    labels.append(f"mlp_{i}")
assert len(labels) == resids_decomp_scaled.shape[0] == resids_accum_scaled.shape[0]

In [167]:
ylim = df_decomp["Logit Diff"].abs().max()
fig = px.line(
    df_decomp,
    title="Logit Lens (Decomposed Residual Stream): logit[dishonest_token] - logit[honest_token]",
    x="Component",
    y="Logit Diff",
    animation_frame="Batch",
    color="Alpha",
    height=600,
)

# Set plot titles and axis labels
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        gridwidth=0.1,
        gridcolor='lightgray',
        dtick=1,
        showline=True,
        linecolor='grey',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black',
        linewidth=1,
        tickmode='array',  # Use an array for tick values
        tickvals=list(range(len(labels))),  # Set tick positions
        ticktext=labels,
        tickangle=-90,
    ),
    
    yaxis=dict(
        showgrid=True,
        gridwidth=0.1,
        gridcolor='lightgray',
        dtick=1,
        showline=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        linewidth=1,
        linecolor='grey',
        tickmode='array',
        tick0=0,
        range=[-ylim, ylim],
    ),
)

# Set background color to white
fig.update_layout(plot_bgcolor='white')

# Show the legend
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

# Fix margins for visibility
fig.update_layout(margin=dict(l=20, r=20, t=100, b=20))
fig['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 110, b=0)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 100,)

# Create red-green color gradient for alpha
plotly_colors = []
for redness in np.linspace(255, 0, 12):
    greenness = 255 - redness
    plotly_colors.append(f"rgb({redness}, {greenness}, 0)")

# Change colors for first frame
for i, color in zip(range(len(alphas)), plotly_colors):
    fig.data[i].line.color = color

# Change colors for all frames
for frame in fig.frames:
    for i, color in zip(range(len(alphas)), plotly_colors):
        frame.data[i].line.color = color

In [169]:
fig.write_html("figs/logit_lens_decomp.html")
# fig.show()

In [170]:
ylim = df_accum["Logit Diff"].abs().max()
fig = px.line(
    df_accum,
    title="Logit Lens (Accumulated Residual Stream): logit[dishonest_token] - logit[honest_token]",
    x="Component",
    y="Logit Diff",
    animation_frame="Batch",
    color="Alpha",
    height=600,
)

# Set plot titles and axis labels
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        gridwidth=0.1,
        gridcolor='lightgray',
        dtick=1,
        showline=True,
        linecolor='grey',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black',
        linewidth=1,
        tickmode='array',  # Use an array for tick values
        tickvals=list(range(len(labels))),  # Set tick positions
        ticktext=labels,
        tickangle=-90,
    ),
    
    yaxis=dict(
        showgrid=True,
        gridwidth=0.1,
        gridcolor='lightgray',
        dtick=1,
        showline=True,
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        linewidth=1,
        linecolor='grey',
        tickmode='array',
        tick0=0,
        range=[-ylim, ylim],
    ),
)

# Set background color to white
fig.update_layout(plot_bgcolor='white')

# Show the legend
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

# Fix margins for visibility
fig.update_layout(margin=dict(l=20, r=20, t=100, b=20))
fig['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 110, b=0)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 100,)

# Create red-green color gradient for alpha
plotly_colors = []
for redness in np.linspace(255, 0, 12):
    greenness = 255 - redness
    plotly_colors.append(f"rgb({redness}, {greenness}, 0)")

# Change colors for first frame
for i, color in zip(range(len(alphas)), plotly_colors):
    fig.data[i].line.color = color

# Change colors for all frames
for frame in fig.frames:
    for i, color in zip(range(len(alphas)), plotly_colors):
        frame.data[i].line.color = color

In [172]:
fig.write_html("figs/logit_lens_accum.html")
# fig.show()